# Eclipse Mapping while Checking if Physical (non-negative maps)
# More realistic NIRCam Sim
Uses standard deviation from mirage simulation at expected real cadence

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry
from copy import deepcopy
import astropy.units as u
import astropy.constants as const
from astropy.table import Table
from astropy.io import fits, ascii
from corner import corner
import os

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
starry.__version__

## Set up Forward Model

Use Earth map as an example map

In [ ]:
max_ell = 3

up_bin = 10 ## up-bin for clarity
lc_precision = 145e-6 / np.sqrt(up_bin) ## measured from broadband F444W precision
cadence = 2.72486 * up_bin ## between exposures
npoints = int(np.round(0.2 * 24. * 3600.  / (cadence))) ## duration/cadence
extra_descrip = '_check_physical_nc_f444w'

map1 = starry.Map(max_ell)

map1.load("earth", sigma=0.05)
y_input = deepcopy(map1.y[1:])
map1.show(projection="rect")

In [ ]:
fig, ax = plt.subplots()
map1.show(projection='ortho',ax=ax)
fig.savefig('plots/forward_model/forward_map{}.png'.format(extra_descrip),dpi=150,bbox_inches='tight')

Make a lightcurve for HD 189733 with this map.

Start by reading in a model for the depth

In [ ]:
modelDat = ascii.read('sim_data/hd189733b_spec_model_binned.csv')
modelDat[1]['depth']

In [ ]:
M_star = 0.812 ## Addison et al. 2019, solar masses
R_star = 0.765 ## Addison et al. 2019, solar radii
inc=85.69 ## Addison et al. 2019
#inc=90.0
rp = 0.1504 * R_star ## Addison et al. 2019, Solar radii
P_b = 2.218577 ## days
log_amp = np.log10(modelDat[1]['depth'])
t0 = 0.0

M_planet = (1.166 * u.Mjup).to(u.Msun).value ## Addison et al. 2019, Solar masses

prot_star = 1.0

x = np.linspace(0.5 * P_b - 0.1,0.5 * P_b + 0.1,npoints)
#x = np.linspace(0.,P_b,4096)


Get the impact parameter

In [ ]:
norb = 2. * np.pi / (P_b * u.day)
a_orb = (const.G * (M_star * u.Msun) / norb**2)**(1./3.)
a_over_r_star = (a_orb/(R_star * u.Rsun)).si
b_impact = a_over_r_star * np.cos(inc * np.pi/180.)
b_impact

In [ ]:
a_orb.to(u.AU)

In [ ]:
a_over_r_star

In [ ]:
y_input

In [ ]:
y_table = Table()
y_labels1, y_values1 = [], []
for one_l in np.arange(max_ell)+1:
    counter=1
    print("Y_{},m=[".format(one_l))
    for one_m in np.arange(-one_l,one_l+1):
        y_labels1.append("$Y_{{{},{}}}$".format(one_l,one_m))
        print("Y_{}_{} = {}".format(one_l,one_m,y_input[counter]))
        #print(y_input[counter])
        counter = counter + 1
    print("] \n")
y_table['label'] = y_labels1
y_table['value'] = np.round(y_input,4)
y_table['res'] = '\\nodata'
y_table.write('plots/forward_model/forward_mod.tex',overwrite=True)

Find the transit durations for contacts 1 to 4 and 2 to 23

In [ ]:
arg = np.sqrt((1. + rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_14 = (P_b / np.pi) * np.arcsin(arg)
Thalf_14 = (0.5 * Tdur_14).value

arg2 = np.sqrt((1. - rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_23 = (P_b / np.pi) * np.arcsin(arg2)
Thalf_23 = (0.5 * Tdur_23).value
Thalf_14,Thalf_23

In [ ]:
A = starry.Primary(starry.Map(ydeg=0, udeg=2, amp=1.0), m=M_star, r=R_star,
                   prot=prot_star )
b = starry.kepler.Secondary(map1,
                            m=M_planet,r=rp,prot=P_b,porb=P_b,t0=t0,inc=inc)
b.map.amp = 10**log_amp

In [ ]:
b.theta0 = 180.0 + 0.0
sys = starry.System(A,b)

In [ ]:
def plot_model_w_data(x,model,data,yerr,ingressZoom=False):
    if ingressZoom == True:
        fig, axArr = plt.subplots(1,2,sharey=True,figsize=(9,4))
        outName = "forward_model_lc{}_zoom.pdf".format(extra_descrip)
    else:
        fig, ax1 = plt.subplots(figsize=(9,4))
        axArr = [ax1]
        outName = "forward_model_lc{}.pdf".format(extra_descrip)
        
    for i,ax in enumerate(axArr):
        ax.set_xlabel("Time (days)")
        if i==0:
            ax.set_ylabel("Normalized Flux")
        ax.errorbar(x,ysim,yerr=yerr,fmt='.',zorder=0)
        ax.plot(x,flux_input,zorder=2)

        
        if ingressZoom == True:
            #ax.set_xlim(0.5 * P_b - Thalf_14,0.5 * P_b - Thalf_23)
            t_ing = Thalf_14 - Thalf_23
            t1 = Thalf_14 + t_ing * 0.2
            t2 = Thalf_23 - t_ing * 0.2
            #t1, t2 = 0.044, 0.0325
            if i==0:
                
                ax.set_xlim(0.5 * P_b - t1,0.5 * P_b -t2)
            else:
                ax.set_xlim(0.5 * P_b + t2,0.5 * P_b + t1)
    savePath = os.path.join('plots','forward_model',outName)
    fig.savefig(savePath)
                
flux_input = deepcopy(sys.flux(x))

np.random.seed(0)
yerr = np.ones_like(x) * lc_precision
ysim = flux_input + np.random.randn(len(x)) * yerr

plot_model_w_data(x,flux_input,ysim,yerr)




In [ ]:
plot_model_w_data(x,flux_input,ysim,yerr,ingressZoom=True)

In [ ]:
def compute_info(A):
    """Compute some information about the null space of the design matrix A."""
    # Get the Fisher information & compute its rank
    I = A.T.dot(A)
    R = np.linalg.matrix_rank(I)

    # Number of coefficientss
    C = I.shape[0]

    # Size of null space
    N = C - R

    # Fractional size of null space
    F = N / C

    # Show it graphically
    fig, ax = plt.subplots(figsize=(6, 0.3))
    ax.set_xlim(0, 1)
    ax.axis("off")
    ax.axvspan(0, 1 - F, color="C0")
    ax.axvspan(1 - F, 1, color="red")
    ax.annotate(
        "{}/{}".format(R, C),
        color="C0",
        fontsize=10,
        xy=(-0.025, 0.5),
        xycoords="axes fraction",
        va="center",
        ha="right",
    )
    ax.annotate(
        "{:.0f}%".format(100 * F),
        color="w",
        fontsize=10,
        xy=(1 - 0.5 * F, 0.5),
        xycoords="axes fraction",
        va="center",
        ha="right",
    )

Check the size of the nullspace

In [ ]:
A = sys.design_matrix(x)
compute_info(A)

In [ ]:
for i in np.arange(6):#sec.map.Ny):
    plt.plot(A[:,i] - 0.6 * i)

For degree=4 there's a pretty small nullspace, but still some coefficints are unconstrained. For degree=3 I don't see any nullspace which is pretty cool!

# Solve the Linear System

In [ ]:
# Prior on primary
# pri_mu = np.zeros(sys.primary.map.Ny)
# pri_mu[0] = 1.0
# pri_L = np.zeros(pri.map.Ny)
# pri_L[0] = 1e-2
# pri_L[1:] = 1e-2
# pri.map.set_prior(mu=pri_mu, L=pri_L)

# Prior on the planet = secondary
sec = sys.secondaries[0]
sec_mu = np.zeros(sec.map.Ny)
sec_mu[0] = 1e-3
## sec_mu[1:] = y_input * sec_mu[0]## what if we cheat at start them at the correct values? Just to check 
## that it can recover
sec_L = np.zeros(sec.map.Ny)
sec_L[0] = (0.2 * sec_mu[0])**2 ## covariance is squared
#sec_L[1:] = (1e-8)**2
#sec_L[1:] = (1.0 * sec_mu[0])**2 ## THIS GIVES MOSTLY UNPHYSICAL (negative)
sec_L[1:] = (0.15 * sec_mu[0])**2 ### THIS GIVES MOSTLY PHYSICAL MAPS (>=0)

sec.map.set_prior(mu=sec_mu, L=sec_L)

In [ ]:
sys.set_data(ysim, C=yerr**2)

In [ ]:
mu, cho_cov = sys.solve(t=x)

In [ ]:
sec.map.amp = mu[0]
sec.map[1:, :] = mu[1:] / sec.map.amp
sys.secondaries[0].map = sec.map

plot_model_w_data(x,sys.flux(x),ysim,yerr,ingressZoom=False)

In [ ]:
plot_model_w_data(x,sys.flux(x),ysim,yerr,ingressZoom=True)

### Check the amplitudes
The solution given gives the Cholesky decomposition of the covariance matrix (not the actual covariance matrix). I think this is how you get the covariance matrix. You can also skip down to the corner plot, where it looks good.

In [ ]:
cov = np.dot(cho_cov,cho_cov.T)
np.sqrt(np.diag(cov))

Compare the amplitudes and spherical harmonics

In [ ]:
print(10**log_amp, mu[0], np.sqrt(cov[0,0]))

mu[1:]/mu[0]

coeff_ind = np.arange(len(y_input)) + 1
coeff_mean = mu[1:]/mu[0]
coeff_err = np.sqrt(np.diag(cov))/mu[0]

coeff_labels = [r"$Y_{%d,%d}$" % (l, m)
    for l in range(1, sec.map.ydeg + 1)
    for m in range(-l, l + 1)
]




fig, ax = plt.subplots(figsize=(12,6))
ax.plot(coeff_ind,y_input,label='Input Forward Model')
ax.errorbar(coeff_ind,coeff_mean,yerr=coeff_err[1:],label='Posterior Mean')
ax.set_xlabel("Coefficient index")
ax.set_ylabel("$C_{l}^m$")
ax.legend()

for ind,oneLabel in enumerate(coeff_labels):
    ax.text(coeff_ind[ind] - 0.3,-0.1 + np.mod(ind,2) * 0.2,oneLabel)
fig.savefig('plots/linear_results/coeff_results_linear_fit{}.pdf'.format(extra_descrip))


### Original Input Map - Dayside

In [ ]:
sec = sys.secondaries[0]
sec.map.amp = 10**log_amp
sec.map.y[1:] = y_input
#sec.map.show(projection='rect')
sec.map.show(projection='ortho',theta=0.0)
#sec.map.show(theta=180.0)

In [ ]:
## Confirm that this is the dayside by visualizing the whole system
#sys.show(0.46 * P_b,figsize=(6,6))

## Mean Map

In [ ]:
# mean values
sec = sys.secondaries[0]

# sec.map.amp = mu[0]
# ## The tutorial notebook divides mu by amp, but it didn't look correct
# sec.map[1:, :] = mu[1 : sec.map.Ny]  #/ sec.map.amp

sec.map.amp = mu[0]
sec.map[1:, :] = mu[1:] / sec.map.amp

sec.map.show(projection="ortho")

In [ ]:
res = sec.map.render(projection='rect')

In [ ]:
np.max(res), np.min(res)

In [ ]:
sec.map.y[1:], y_input

Check the residuals

In [ ]:
sys.secondaries[0].map = sec.map
yfit = sys.flux(x)
resid = yfit - ysim

fig, (ax,ax2) = plt.subplots(2,sharex=True)
ax.plot(x,yfit)
ax.errorbar(x,ysim,alpha=0.5)
ax2.plot(x,resid)

Compare a few draws with the truth

In [ ]:
np.random.seed(4)
for i in np.arange(5):
    sys.draw()
    sys.secondaries[0].map.show(projection='ortho')

In [ ]:
truths = np.append(10**log_amp,y_input[:8])

nsamples = 10000
u = np.random.randn(len(mu), nsamples)
samples = mu.reshape(1, -1) + np.dot(cho_cov, u).T

# De-weight the samples to get
# samples of the actual Ylm coeffs
samps = np.array(samples[:, :9])
samps[:, 1:] /= samps[:, 0].reshape(-1, 1)

In [ ]:
fig, ax = plt.subplots(9, 9, figsize=(12, 12))
labels = [r"$\alpha$"] + [
    r"$Y_{%d,%d}$" % (l, m)
    for l in range(1, sec.map.ydeg + 1)
    for m in range(-l, l + 1)
]

corner(samps, fig=fig, labels=labels,truths=truths)
for axis in ax.flatten():
    axis.xaxis.set_tick_params(labelsize=6)
    axis.yaxis.set_tick_params(labelsize=6)
    axis.xaxis.label.set_size(12)
    axis.yaxis.label.set_size(12)
    axis.xaxis.set_label_coords(0.5, -0.6)
    axis.yaxis.set_label_coords(-0.6, 0.5)
fig.savefig('plots/corner/Flat_001_maptype_variable_amp_type_variableFlat{}.png'.format(extra_descrip))

# Ensure Physical Results
Check what fraction of the samples are physical (non-negative flux)

In [ ]:
n_samples = nsamples
map_mins = []
sec_map = sys.secondaries[0].map
for i in np.arange(n_samples):
    #sys.draw()
    #sys.secondaries[0].map.y[1:] = samples[i,1:]
    sec_map.amp = samples[i,0]
    sec_map.y[1:] = samples[i,1:] / samples[i,0]
    map_evaluate = sec_map.render(projection='rect',res=100)
    map_mins.append(np.min(map_evaluate))

pos_maps = np.array(map_mins) > 0
n_pos = np.sum(pos_maps)
print("{} out of {} are positive".format(n_pos,n_samples))

See how much better the precision is on these coefficients

In [ ]:
good_samples = samples[pos_maps,:]

good_coeff = good_samples[:,1:] / np.tile(good_samples[:,0],[len(coeff_ind),1]).T
mean_good = np.mean(good_coeff,axis=0)
median_good = np.median(good_coeff,axis=0)
stdev_good = np.std(good_coeff,axis=0)
#    sec_map.y[1:] = good_samples[i,1:] / good_samples[i,0]
#coeff_mean_new = np.mean()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(coeff_ind,y_input,label='Input Forward Model')
ax.errorbar(coeff_ind,coeff_mean,yerr=coeff_err[1:],label='Posterior Mean, All',capsize=3)
ax.errorbar(coeff_ind,median_good,yerr=stdev_good,label='Posterior Mean, Physical',capsize=3)
ax.set_xlabel("Coefficient index")
ax.set_ylabel("$C_{l}^m$")
ax.legend()

for ind,oneLabel in enumerate(coeff_labels):
    ax.text(coeff_ind[ind] - 0.3,-0.2 + np.mod(ind,2) * 0.2,oneLabel)



Except for $Y_{1,0}$, this looks way better

## Plot the maps that are positive

In [ ]:
nshow = 4
sec_map = sys.secondaries[0].map

for i in np.arange(nshow):
    sec_map.amp = good_samples[i,0]
    sec_map.y[1:] = good_samples[i,1:] / good_samples[i,0]
    sec_map.show(projection='ortho')

These are much closer to the input map!

# Add a quadratic (2nd order) Baseline

In [ ]:
x_norm = (x - np.mean(x))/(0.5 * (np.max(x) - np.min(x)))
c = np.array([2e-4,3e-4,1.]) ## backwards order c_0 x^n + c_(1) x^(n-1) + ... c_(n-2) x^2 + c_(n-1) x + 1
baseline = np.polyval(c,x_norm)

bsim = ysim * baseline

plt.plot(x,bsim)
plt.xlabel('Time (JD)')
plt.ylabel("Normalized Flux")

## save the baseline

In [ ]:
outTable = Table()
outTable['Time (days)'] = x
outTable['Flux'] = bsim
outTable['Flux err'] = yerr
outTable['Flux before Baseline'] = ysim
outTable['Baseline'] = baseline
meta1 = {'Amplitude':10**log_amp,
         'Period':P_b,
         'Period_units': 'days',
         'M_star': M_star,
         'M_star_units': 'Msun',
         'R_star': R_star,
         'M_planet': M_planet,
         'M_planet_units': 'Msun',
         'inc': inc,
         'inc_units': 'deg',
         'rp': rp,
         'rp_units': 'Rsun',
         't0': t0,
         'y_input': list(y_input),
         'baseline_c': list(c),
         'dur_14': Thalf_14 * 2.,
         'dur_23': Thalf_23 * 2.,
         'b_impact': b_impact,
         'prot_star': prot_star}
outTable.meta = meta1
outTable.write('sim_data/sim_data_baseline_hd189_ncF444W.ecsv',overwrite=True)

Save the noiseless and noisy model

In [ ]:

fig, ax = plt.subplots(1,sharex=True)
ax.plot(x,flux_input,label='Noiseless Model',zorder=2,linewidth=2,color='black')
ax.plot(x,ysim,'.',label='Simulated Data: Flat Baseline',zorder=1,color='green')
ax.plot(x,bsim,'.',label='Simulated Data: Baseline Trend',zorder=1,color='red')
ax.set_ylabel("Normalized Flux")
ax.set_xlabel("Time (days)")
ax.legend()
fig.savefig('plots/forward_model/forward_model_w_baseline_hd189_f444w.pdf',bbox_inches='tight')